In [1]:
#import libraries
import numpy as np
import cv2 
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
def convLayer(X,num_input_channels,num_filters,filter_shape,pool_shape,name):

    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                      num_filters]
    b1 = tf.Variable(tf.zeros([num_filters]))
    filter1 = tf.get_variable("filter"+name,shape=conv_filt_shape,dtype = tf.float32,initializer = tf.contrib.layers.xavier_initializer_conv2d())
    conv1 = tf.nn.conv2d(X,filter1,strides=[1,1,1,1],padding = "SAME")
    conv1 += b1
    conv1 = tf.nn.relu(conv1)
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    pool1 = tf.nn.max_pool(conv1,ksize=ksize,strides=strides,padding="SAME")
    return pool1

def faceDetect(img, face_classifier):
    img_faces=[]
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=3);
    return faces

In [3]:
labels=['Asjad','Fahad','Minahil','Umer']
total_classes = len(labels)

tf.reset_default_graph()
learning_rate = tf.placeholder(tf.float32,[])
X = tf.placeholder(tf.float32,[None,200,200,3])
Y = tf.placeholder(tf.float32,[None,total_classes])
conv1 = convLayer(X,3,64,[3,3],[2,2],"1")
print (conv1.shape)
conv2 = convLayer(conv1,64,64,[3,3],[2,2],"2")
print (conv2.shape)
conv3 = convLayer(conv2,64,32,[3,3],[2,2],"3")
print (conv3.shape)
conv4 = convLayer(conv3,32,16,[3,3],[2,2],"4")
print (conv4.shape)
conv5 = convLayer(conv4,16,8,[3,3],[2,2],"5")
print (conv5.shape)
flattened = tf.reshape(conv5,[-1,7*7*8])

dense1 = tf.layers.dense(flattened,total_classes)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = dense1,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

a = tf.argmax(dense1,axis=1)
b = tf.argmax(Y,axis=1)
prediction = tf.equal(a,b)
accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))

saver = tf.train.Saver()

(?, 100, 100, 64)
(?, 50, 50, 64)
(?, 25, 25, 32)
(?, 13, 13, 16)
(?, 7, 7, 8)


In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 400) #WIDTH
cap.set(4, 600) #HEIGHT

face_cascade = cv2.CascadeClassifier('C:/Users/Fahad/Anaconda2/envs/mypython35/Library/etc/haarcascades/haarcascade_frontalface_alt.xml')
sess=tf.Session()
saver.restore(sess,'./Models/best_model7.0.ckpt')
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceDetect(frame, face_cascade)
    # Display the resulting frame
    for x,y,w,h in faces:
         cv2.rectangle(frame,(x,y),(x+w,y+h),(173,255,51),2)
         colored_face = frame[y:y+h, x:x+w]
         colored_face = cv2.resize(colored_face,(200,200))
         
    values=sess.run(dense1,feed_dict={X:colored_face[np.newaxis,:,:,:]})
    value=np.argmax(values)
    score_array=np.array((value,value,value,value))
    score=np.sum(score_array-values)
    if (score*10)<100:
        cv2.putText(frame,"Name: "+labels[value],(x,y-5), 2, 1,(173,255,51),2,cv2.LINE_AA)
        cv2.putText(frame,"Score: "+str(int(abs(score*10))),(50,450), 2, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.flip(frame,0)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from ./Models/best_model7.0.ckpt
